# Run ollama in a single Colab Session

To enable GPU in this notebook, select Runtime -> Change runtime type in the Menu bar. Under Hardware Accelerator, select GPU.

Then, scroll to the Configuration [cell](#scrollTo=8WIVDY-V-kVw&line=1&uniqifier=1) and update it with your ngrok authentication token.

To run, select Runtime -> Run all. Go to this [cell](#scrollTo=5M_2NlAXB89F&line=1&uniqifier=1) and read the instructions on how to update your `.env` file.

# Check GPU

In [ ]:
!nvidia-smi

# Install Deps

In [ ]:
!apt update -y -qq
!apt install -y -qq curl lshw aria2 libcairo2-dev pkg-config python3-dev
!curl https://ollama.ai/install.sh | sh

!pip install flask -q
!pip install pyngrok -q
!pip install requests -q
!pip install flask-cors -q
import subprocess

sub = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE)
!ollama pull mistral:instruct

# Configuration

Please set the NGROK auth token to access the tunnel.

In [ ]:
NGROK_AUTH_TOKEN = '2NPZbjWnWpqUallvFrDj3Q6z8MQ_32PYSNp438wFuskyvMs9M' #@param {type:'string'}
# TRANSLATE_FILENAME = 'translate.wav' #@param {type:'string'}
# TRANSCRIBE_FILENAME = 'transcribe.wav' #@param {type:'string'}
# WHISPER_MODEL = 'small' #@param ['tiny', 'base', 'small', 'medium', 'large']
# VOICEVOX_URL = 'http://localhost:50021' #@param {type:'string'}
# CHUNK_SIZE = 4096 #@param {type:'integer'}
OLLAMA_URL = 'http://127.0.0.1:11434' #@param {type:'string'}

# Main Code

In [ ]:
from flask import Flask, request, Response
import json
import subprocess
import requests
from pyngrok import ngrok
from flask_cors import CORS
import os
from urllib.parse import urlencode

In [ ]:
app = Flask(__name__)
ngrok.set_auth_token(NGROK_AUTH_TOKEN)
CORS(app)

@app.route('/', methods=['GET'])
def test():
  response = {'status':'OK','message':''}
  return Response(json.dumps({'status':'OK','message':'Test'}), mimetype='application/json')

@app.route('/api/generate', methods=['POST']) # Create route for generate_completion function
def generate_completion():
  model = request.form.get('model') # Get value model from the parameter url
  prompt = request.form.get('prompt') # Get value prompt from the parameter url
  json_data = { # JSON data to be sent in the POST request
    "model": model or 'mistral:instruct',
    "prompt": prompt,
    "system": "You are 2B from NieR Automata. Answer as 2B, the assistant, only.",
    "temperature": 0.8,
    "stream": False
  }
  headers = { "Content-Type": "application/json" } # Set the headers for the request
  response = requests.post(f'{OLLAMA_URL}/api/generate', json=json_data, headers=headers)
  return json.loads(response.text)['response']


def main():
  sub = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE)

  http_tunnel = ngrok.connect(5000) # Open tunnel
  print(http_tunnel)

  app.run() # Run app

if __name__ == '__main__':
  try:
    main()
  except KeyboardInterrupt:
    print('ded')